In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys
import pandas as pd
import math
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import json

In [2]:
data = pd.read_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/Final Res.csv')
data
# 카페 데이터만
data = data.loc[data['상권업종중분류명'] == '커피점/카페']  
data

,Unnamed: 0,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,...,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도,역명,거리,카카오url
7,66,28524822,빈트리망원점,망원점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,121230,4007.0,NaN,1,NaN,126.899339,37.556041,9호선 선유도,1294.649872,NaN
13,137,15520570,더블유카페,역삼2호점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,135921,6225.0,NaN,1,2,127.043902,37.499426,9호선 선정릉,764.611819,NaN
15,147,22646116,커피빈,코리아선릉역점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,135080,6150.0,NaN,1,NaN,127.048223,37.504750,9호선 삼성중앙,461.237983,NaN
29,222,25437483,카페그리다,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,135090,6159.0,NaN,1,NaN,127.052597,37.506767,9호선 삼성중앙,554.451907,NaN
35,274,23739181,홈플러스어라운드홈,강서점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,157030,7567.0,NaN,1,NaN,126.854800,37.558456,9호선 가양,236.335955,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12774,135527,17249536,티컬렉티브,삼성점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,135868,6097.0,NaN,NaN,NaN,127.048787,37.511981,9호선 삼성중앙,35.645121,NaN
12782,135625,17273431,트윗젤,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,135876,6212.0,220,NaN,116,127.048927,37.504494,9호선 삼성중앙,478.709107,NaN
12784,135632,17254243,후르츠13.5,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,157805,7788.0,NaN,12,NaN,126.825403,37.568078,9호선 마곡나루,235.823443,NaN
12785,135633,17252972,카페,MJ보드,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,157805,7788.0,NaN,13,NaN,126.827682,37.568569,9호선 마곡나루,135.140328,NaN


In [3]:
data = data[["상호명","상권업종중분류명","상권업종소분류명","도로명주소","역명","거리","카카오url"]]
data["역"] = data["역명"].apply(lambda x : x.split(" ")[1])
data["역"] = data["역"].apply(lambda x : x.split("(")[0])
data["검색어"] = data.apply(lambda x : x["역"] + "역 " + str(x["상호명"]),axis=1)
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,카카오url,역,검색어
7,빈트리망원점,커피점/카페,커피전문점/카페/다방,서울특별시 마포구 망원로 22,9호선 선유도,1294.649872,NaN,선유도,선유도역 빈트리망원점
13,더블유카페,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 언주로 407,9호선 선정릉,764.611819,NaN,선정릉,선정릉역 더블유카페
15,커피빈,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 테헤란로 335,9호선 삼성중앙,461.237983,NaN,삼성중앙,삼성중앙역 커피빈
29,카페그리다,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 테헤란로69길 14,9호선 삼성중앙,554.451907,NaN,삼성중앙,삼성중앙역 카페그리다
35,홈플러스어라운드홈,커피점/카페,커피전문점/카페/다방,서울특별시 강서구 화곡로 398,9호선 가양,236.335955,NaN,가양,가양역 홈플러스어라운드홈
...,...,...,...,...,...,...,...,...,...
12774,티컬렉티브,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 봉은사로 449,9호선 삼성중앙,35.645121,NaN,삼성중앙,삼성중앙역 티컬렉티브
12782,트윗젤,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 테헤란로 340,9호선 삼성중앙,478.709107,NaN,삼성중앙,삼성중앙역 트윗젤
12784,후르츠13.5,커피점/카페,커피전문점/카페/다방,서울특별시 강서구 마곡서로 158,9호선 마곡나루,235.823443,NaN,마곡나루,마곡나루역 후르츠13.5
12785,카페,커피점/카페,커피전문점/카페/다방,서울특별시 강서구 마곡중앙로 165,9호선 마곡나루,135.140328,NaN,마곡나루,마곡나루역 카페


In [4]:
items = data["검색어"]
items

7          선유도역 빈트리망원점
13          선정릉역 더블유카페
15           삼성중앙역 커피빈
29         삼성중앙역 카페그리다
35       가양역 홈플러스어라운드홈
             ...      
12774      삼성중앙역 티컬렉티브
12782        삼성중앙역 트윗젤
12784    마곡나루역 후르츠13.5
12785         마곡나루역 카페
12799        노량진역 카페시모
Name: 검색어, Length: 2215, dtype: object

In [5]:
rating_df = pd.DataFrame(columns = ["UserID","ItemID","Rating","Timestamp"])
img_elem = pd.DataFrame(columns = ["ItemID","img_url"])
review_elem = pd.DataFrame(columns = ["ItemID", "review"])
user_review_id = {}
review_json = {}
image_json = {}

In [ ]:
# 드라이버 구동
driver = webdriver.Chrome(executable_path='C:/Users/정예림/Desktop/추천시스템 프로젝트/chromedriver.exe')
# 창 크기 최대화
driver.maximize_window() 



count = 0
current = 0
indexing = 0
goal = len(items)
for item in items :
    current += 1
    print('진행상황 : ', current,'/',goal,sep="")
    # 리뷰가 없을 때의 코드
    driver.get("https://map.kakao.com/") # 카카오 지도 접속하기
    searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']") # 검색창에 입력하기
    searchbox.send_keys(item)
    time.sleep(2)
    searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") # 검색버튼 누르기
    driver.execute_script("arguments[0].click();", searchbutton)
    time.sleep(2)
    
    if len(driver.find_elements_by_xpath("//a[@class='moreview']")) != 0:
        print('식당 존재')
        driver.execute_script('window.open("about:blank", "_blank");') # 새 탭 열기
        reviewbutton = driver.find_element_by_xpath("//a[@class='numberofscore']")
        time.sleep(2)
        content_url = reviewbutton.get_attribute("href") 
        #상세보기 URL수집
        url = driver.find_element_by_xpath('//*[@id="info.search.place.list"]/li[1]/div[5]/div[4]/a[1]').get_attribute('href')
        data.iloc[indexing,-3] = url
        print(url)
        tabs = driver.window_handles
        driver.switch_to_window(tabs[1]) # 새 탭으로 이동
        driver.get(content_url) # 링크 열기
        time.sleep(3)
        html = driver.page_source
        time.sleep(20)
        soup = BeautifulSoup(html, 'html.parser')
        review_lists = soup.select('.list_evaluation > li')
        print(len(review_lists))
        if len(review_lists) != 0 :
            for i, review in enumerate(review_lists) :
                user_review = review.select('.txt_comment > span') # 리뷰
                rating = review.select('.grade_star > em') # 별점
                img_url = review.select('.link_photo > img') #이미지url
                user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                timestamp = review.select(' div > span.time_write') #시간정보
                try:
                    row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_ur}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)

                    time.sleep(1)
                except:
                    row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_url}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    time.sleep(1)

                
                    
        else :
            print("리뷰가 없습니다")
            
        try:
            for i in range(2,500):
                time.sleep(3)
                another_review = driver.find_element_by_xpath("//a[@data-page='" + str(i) + "']")
                another_review.click()
                time.sleep(3)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                review_lists = soup.select('.list_evaluation > li')
                if len(review_lists) != 0 :
                    for i, review in enumerate(review_lists) :
                        user_review = review.select('.txt_comment > span') # 리뷰
                        rating = review.select('.grade_star > em') # 별점
                        img_url = review.select('.link_photo > img') #이미지url
                        user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                        timestamp = review.select(' div > span.time_write') #시간정보
                        try:
                            row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                            
                        except:
                            row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                        
                        
                            
        except:
            print("더 이상 리뷰 존재 X")
            driver.close()
        driver.switch_to_window(tabs[0])
        print("기본 페이지로 돌아가자")
            
    else:
        data.iloc[indexing,-3] = None
        print("식당 존재 x")
    
    indexing += 1


진행상황 : 1/2215
식당 존재 x
진행상황 : 2/2215
식당 존재
https://place.map.kakao.com/22646026
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 3/2215
식당 존재 x
진행상황 : 4/2215
식당 존재
https://place.map.kakao.com/321704063
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 5/2215
식당 존재 x
진행상황 : 6/2215
식당 존재
https://place.map.kakao.com/21405265
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 7/2215
식당 존재
https://place.map.kakao.com/1636263278
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 8/2215
식당 존재 x
진행상황 : 9/2215
식당 존재 x
진행상황 : 10/2215
식당 존재 x
진행상황 : 11/2215
식당 존재
https://place.map.kakao.com/639700545
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 12/2215
식당 존재
https://place.map.kakao.com/1522483669
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 13/2215
식당 존재
https://place.map.kakao.com/860472511
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 14/2215
식당 존재 x
진행상황 : 15/2215
식당 존재
https://place.map.kakao.com/24765625
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 16/2215
식당 존재
https://place.map.kakao.com/1862570607
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 17/2215
식당 존재 x
진행상황 :

식당 존재 x
진행상황 : 135/2215
식당 존재 x
진행상황 : 136/2215
식당 존재
https://place.map.kakao.com/1529498058
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 137/2215
식당 존재
https://place.map.kakao.com/8438452
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 138/2215
식당 존재 x
진행상황 : 139/2215
식당 존재
https://place.map.kakao.com/21532642
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 140/2215
식당 존재 x
진행상황 : 141/2215
식당 존재
https://place.map.kakao.com/440142287
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 142/2215
식당 존재 x
진행상황 : 143/2215
식당 존재
https://place.map.kakao.com/14712077
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 144/2215
식당 존재
https://place.map.kakao.com/668868626
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 145/2215
식당 존재
https://place.map.kakao.com/10555584
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 146/2215
식당 존재
https://place.map.kakao.com/2052340190
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 147/2215
식당 존재
https://place.map.kakao.com/1943853072
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 148/2215
식당 존재 x
진행상황 : 149/2215
식당 존재
https://place.map.kakao.com/18202705
0
리뷰가 없습니다
더 이

5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 261/2215
식당 존재 x
진행상황 : 262/2215
식당 존재 x
진행상황 : 263/2215
식당 존재
https://place.map.kakao.com/1612787942
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 264/2215
식당 존재
https://place.map.kakao.com/49050456
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 265/2215
식당 존재
https://place.map.kakao.com/26338066
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 266/2215
식당 존재 x
진행상황 : 267/2215
식당 존재
https://place.map.kakao.com/13057629
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 268/2215
식당 존재 x
진행상황 : 269/2215


In [7]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,7t5a08,방화역 버블베어,4점,2017.12.29.
1,e8ro6t,광나루역 뉴욕핫도그앤커피,3점,2020.04.07.
2,fkl5ls,신정역 커피니,5점,2017.04.26.
3,None,신정역 커피니,None,2017.06.17.
4,None,신정역 커피니,None,2017.06.17.
...,...,...,...,...
1020,7mqvc7,우장산역 할리스커피,4점,2018.05.19.
1021,mvub1p,방화역 빽다방방화사거리점,1점,2020.08.05.
1022,14mold1,방화역 빽다방방화사거리점,1점,2020.05.01.
1023,7m278d,영등포시장역 스타벅스,5점,2021.05.01.


In [8]:
img_elem

,ItemID,img_url
0,방화역 버블베어,None
1,광나루역 뉴욕핫도그앤커피,None
2,신정역 커피니,None
3,신정역 커피니,None
4,신정역 커피니,None
...,...,...
1019,우장산역 할리스커피,None
1020,우장산역 할리스커피,None
1021,방화역 빽다방방화사거리점,None
1022,방화역 빽다방방화사거리점,None


In [9]:
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,카카오url,역,검색어
7,406번째스토브,커피점/카페,커피전문점/카페/다방,서울특별시 양천구 오목로 329-6,5호선 오목교(목동운동장앞),42.783155,NaN,오목교,오목교역 406번째스토브
8,우성커피숍,커피점/카페,커피전문점/카페/다방,서울특별시 양천구 오목로 48,5호선 까치산,794.156052,NaN,까치산,까치산역 우성커피숍
10,버블베어,커피점/카페,커피전문점/카페/다방,서울특별시 강서구 금낭화로 167,5호선 방화,261.092137,https://place.map.kakao.com/18553005,방화,방화역 버블베어
55,리버스로스터,커피점/카페,커피전문점/카페/다방,서울특별시 양천구 가로공원로 71,5호선 마곡,1658.491978,None,마곡,마곡역 리버스로스터
61,필링,커피점/카페,커피전문점/카페/다방,서울특별시 종로구 수표로 110,5호선 종로3가(탑골공원),125.480924,https://place.map.kakao.com/22025606,종로3가,종로3가역 필링
...,...,...,...,...,...,...,...,...,...
20025,세라젬웰카페,커피점/카페,커피전문점/카페/다방,서울특별시 영등포구 영등포로 210,5호선 영등포시장,215.460909,NaN,영등포시장,영등포시장역 세라젬웰카페
20026,미타임,커피점/카페,커피전문점/카페/다방,서울특별시 성동구 천호대로 346,5호선 답십리,587.455177,NaN,답십리,답십리역 미타임
20027,휴식상점,커피점/카페,커피전문점/카페/다방,서울특별시 동대문구 한천로6길 47-2,5호선 장한평,290.438891,NaN,장한평,장한평역 휴식상점
20043,단백한하우스,커피점/카페,커피전문점/카페/다방,서울특별시 송파구 문정로11길 12,5호선 개롱,357.124389,NaN,개롱,개롱역 단백한하우스


In [10]:
img_elem

,ItemID,img_url
0,방화역 버블베어,None
1,광나루역 뉴욕핫도그앤커피,None
2,신정역 커피니,None
3,신정역 커피니,None
4,신정역 커피니,None
...,...,...
1019,우장산역 할리스커피,None
1020,우장산역 할리스커피,None
1021,방화역 빽다방방화사거리점,None
1022,방화역 빽다방방화사거리점,None


In [11]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,7t5a08,방화역 버블베어,4점,2017.12.29.
1,e8ro6t,광나루역 뉴욕핫도그앤커피,3점,2020.04.07.
2,fkl5ls,신정역 커피니,5점,2017.04.26.
3,None,신정역 커피니,None,2017.06.17.
4,None,신정역 커피니,None,2017.06.17.
...,...,...,...,...
1020,7mqvc7,우장산역 할리스커피,4점,2018.05.19.
1021,mvub1p,방화역 빽다방방화사거리점,1점,2020.08.05.
1022,14mold1,방화역 빽다방방화사거리점,1점,2020.05.01.
1023,7m278d,영등포시장역 스타벅스,5점,2021.05.01.


In [12]:
review_elem

,ItemID,review
0,방화역 버블베어,커피는 맛있는데 가격에비해 양이 적어요 ㅜㅜ
1,광나루역 뉴욕핫도그앤커피,보통이에요
2,신정역 커피니,최고예요
3,신정역 커피니,좋아요
4,신정역 커피니,보통이에요
...,...,...
1019,우장산역 할리스커피,쿠폰 사서 시켰더니 다른거 주면서 이미 만들어서 안바꿔 준다니 뭔 말인지 서비스 엉망임
1020,우장산역 할리스커피,"2층은 파스쿠찌랑 붙어있고, 옆에 이디야까지 들어오면서 여기도 좀 카페 전쟁터 느낌..."
1021,방화역 빽다방방화사거리점,주문먼저빼지않고 하나주문받고 하나만들어요. 어느카페가도 이렇게 주문할려고 열심히 줄...
1022,방화역 빽다방방화사거리점,영업시간을 수정하시든지..이건 뭐 22:30이라고 되어 있는데 21시전에 가도 오더...


In [13]:
img_elem.head()

,ItemID,img_url
0,방화역 버블베어,None
1,광나루역 뉴욕핫도그앤커피,None
2,신정역 커피니,None
3,신정역 커피니,None
4,신정역 커피니,None


In [16]:
ItemID_elem = rating_df['ItemID'].unique()

for k in range(len(ItemID_elem)) :
    User_ID_elem = rating_df.loc[rating_df['ItemID'] == ItemID_elem[k]]
    Uder_ID_elem = User_ID_elem[['UserID','ItemID']]
print(User_ID_elem)





ItemID_elem = img_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_img_elem = img_elem.loc[img_elem['ItemID'] == ItemID_elem[j]]
print(item_img_elem)



ItemID_elem = review_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_review_elem = review_elem.loc[review_elem['ItemID'] == ItemID_elem[j]]
print(item_review_elem)




      UserID       ItemID Rating    Timestamp
1023  7m278d  영등포시장역 스타벅스     5점  2021.05.01.
1024    None  영등포시장역 스타벅스   None  2021.04.01.
           ItemID img_url
1023  영등포시장역 스타벅스    None
           ItemID review
1023  영등포시장역 스타벅스   최고예요


In [17]:
rating_df.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_cafe(263).csv')

In [18]:
User_ID_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_cafe_User(263).csv')
item_img_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_cafe_img(263).csv')
item_review_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/lline5_cafe_review(263).csv')

In [19]:
ItemID_elem = rating_df['ItemID'].unique()
len(ItemID_elem)


109

In [20]:
data.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_(1)263url.csv')